# Oblivious Amplitude Amplification

Here, we implement the "oblivious" version of Grover's algorithm. We suppose $|a\rangle$ and $|b\rangle$ are two kets in a possibly large Hilbert space $\mathcal{H}$. We are given access to the unitaries $e^{i\phi}|a\rangle\langle a|$, $e^{i\phi}|b\rangle\langle b|$ for arbitrary $\phi$, as well as a fixed unitary $U$ for which all we know is that $\langle a|U|b\rangle \neq 0$. The aim of the algorithm is to construct, using these resources, a unitary $Q$ such that $|\langle a|Q|b\rangle| \approx 1$.

For our purposes, we consider a system of six qubits where $|a\rangle$ is a superposition of all strings with random nonzero weights, and $|b\rangle$ is the particular string $|111111\rangle$.

In [1]:
'''Some generic python and jupyter imports'''
import numpy as np
from numpy import pi,sqrt

'''Qiskit imports'''
from qiskit import(
    QuantumCircuit
    , execute
    , Aer
    , ClassicalRegister
    , QuantumRegister
)
backend_svec = Aer.get_backend('statevector_simulator')
backend_qasm = Aer.get_backend('qasm_simulator')

'''Visualizations'''
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'qiskit'